In [1]:
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
import scipy
import os
import cv2
import csv
import random
import shutil
from scipy.io import loadmat

In [9]:
width = 640
height = 480
    
foffset = 20

def parseLabel(labelInput):
    #parsed: name tlx tly brx bry ftlx ftly
    #output: name minx miny maxx maxy, fminx, fminy, fmaxx, fmaxy
    
    result = labelInput[0].split('    ')
    name = os.path.splitext(result[0])[0]
    tlx, tly, brx, bry = result[1], result[2], result[3], result[4];
    minx, miny, maxx, maxy = int(float(tlx)*width), int(float(tly)*height), \
                             int(float(brx)*width), int(float(bry)*height)
    
    ftlx, ftly = result[5], result[6];
    fminx, fminy, fmaxx, fmaxy = int(float(ftlx)*width) - foffset, \
                                 int(float(ftly)*height) - foffset, \
                                 int(float(ftlx)*width) + foffset, \
                                 int(float(ftly)*height) + foffset
    
    hand_width = maxx - minx
    hand_height = maxy - miny
    
    fminx = max(0, fminx - minx)
    fminy = max(0, fminy - miny)
    fmaxx = min(hand_width, fmaxx - minx)
    fmaxy = min(hand_height, fmaxy - miny)
    return name, minx, miny, maxx, maxy, fminx, fminy, fmaxx, fmaxy, hand_width, hand_height
            

def cmp_img(a, b):
    a = os.path.splitext(a)[0].split('_')
    b = os.path.splitext(b)[0].split('_')
    
    a_body, a_num = a[1], a[2]
    b_body, b_num = b[1], b[2]
    
    if a_body > b_body:
        return 1
   
    if a_body < b_body:
        return -1
    
    if int(a_num) > int(b_num):
        return 1
    
    if int(a_num) < int(b_num):
        return -1
    
    return 0

def crop(img, xmin, ymin, xmax, ymax):
    return img[ymin:ymax, xmin:xmax]

In [13]:
labelInputs = iter(sorted(['I_Avenue', 'I_BasketballField', 'I_Chinesebook', 'I_ClassRoom', 'I_ComputerScreen']))
currentLabel = labelInputs.next()

data_type = 'train'

imgList = os.listdir('images/%s' % data_type)
imgList.sort(cmp=cmp_img)
print imgList[:20]
totalImg = len(imgList)
print 'total images: %s' % totalImg


labelStream = open('images/label/%s_label.txt' % currentLabel, 'r')
reader = csv.reader(labelStream)

['I_Avenue_0.jpg', 'I_Avenue_1.jpg', 'I_Avenue_3.jpg', 'I_Avenue_4.jpg', 'I_Avenue_5.jpg', 'I_Avenue_7.jpg', 'I_Avenue_8.jpg', 'I_Avenue_9.jpg', 'I_Avenue_10.jpg', 'I_Avenue_11.jpg', 'I_Avenue_12.jpg', 'I_Avenue_13.jpg', 'I_Avenue_14.jpg', 'I_Avenue_15.jpg', 'I_Avenue_16.jpg', 'I_Avenue_17.jpg', 'I_Avenue_18.jpg', 'I_Avenue_19.jpg', 'I_Avenue_20.jpg', 'I_Avenue_21.jpg']
total images: 16114


In [14]:
with open('images/finger_track_%s.csv' % data_type, 'wb') as fout:
    writer= csv.writer(fout)
    writer.writerow(['filename','width','height','class','xmin','ymin','xmax','ymax'])
        
    counter = 0;
    for item in imgList:
        while not item.startswith(currentLabel):
            currentLabel = labelInputs.next()
            labelStream.close()
            labelStream = open('images/label/%s_label.txt' % currentLabel, 'r')
            reader = csv.reader(labelStream)

        name, minx, miny, maxx, maxy, fminx, fminy, fmaxx, fmaxy, hand_width, hand_height = parseLabel(reader.next())
        try:
            while not os.path.splitext(name)[0] == os.path.splitext(item)[0]:
                name, minx, miny, maxx, maxy, fminx, fminy, fmaxx, fmaxy, hand_width, hand_height = parseLabel(reader.next())
        except StopIteration as e:
            print e
            print name
            print item
            raise
            
        counter += 1
        if counter % 1000 == 0:
            print counter

        img = cv2.imread('images/%s/%s' % (data_type, item), 1);
        hand_img = crop(img, minx, miny, maxx, maxy)
        cv2.imwrite('images/%s_finger/%s.jpg' % (data_type, os.path.splitext(name)[0]), 
                                                 hand_img)
        
        finger_img = crop(hand_img, fminx, fminy, fmaxx, fmaxy)
        cv2.imwrite('images/%s_finger_pic/%s.jpg' % (data_type, os.path.splitext(name)[0]), 
                                                     finger_img)

        writer.writerow([item, hand_width, hand_height, 'finger', fminx, fminy, fmaxx, fmaxy])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
